In [2]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
import vertexai

class StreamingChatMemory:
    def __init__(self, model="gemini-1.5-flash-001"):
        # Initialize Vertex AI
        vertexai.init(project='gen-lang-client-0811883729', location='us-central1')
        
        # Create Language Model with streaming
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-pro",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2
        )
        
        # Initialize memory
        self.memory = ConversationBufferMemory(return_messages=True)
    
    def stream_chat(self, user_input):
        # Prepare conversation history
        chat_history = self.memory.chat_memory.messages
        
        # Combine chat history with new input
        messages = chat_history + [HumanMessage(content=user_input)]
        
        # Stream response
        print("AI: ", end='', flush=True)
        full_response = ""
        
        for chunk in self.llm.stream(messages):
            # Extract content from AIMessageChunk
            text_chunk = chunk.content if hasattr(chunk, "content") else str(chunk)
            print(text_chunk, end='', flush=True)  # Print chunk to console
            full_response += text_chunk  # Concatenate chunk to the full response
        
        print()  # New line after response
        
        # Save context to memory
        self.memory.save_context(
            {"input": user_input}, 
            {"output": full_response}
        )
        
        return full_response

    
    def interactive_chat(self):
        print("Streaming Chat Started. Type 'exit' to end.")
        
        while True:
            # Get user input
            user_input = input("You: ")
            print(f"Me : " + user_input)
            # Exit condition
            if user_input.lower() in ['exit', 'quit', 'bye']:
                break
            
            # Stream and memorize response
            self.stream_chat(user_input)
    
    # Unneccesary function generated by a Gen AI
    def print_history(self):
        # Print conversation history
        print("\n--- Conversation History ---")
        for msg in self.memory.chat_memory.messages:
            if isinstance(msg, HumanMessage):
                print(f"You: {msg.content}")
            elif isinstance(msg, AIMessage):
                print(f"AI: {msg.content}")

# Run the chat
def main():
    chat = StreamingChatMemory()
    
    try:
        chat.interactive_chat()
    finally:
        # Print conversation history at the end
        # chat.print_history()
        print("Conversation Ended")

if __name__ == "__main__":
    main()

/opt/homebrew/Caskroom/miniconda/base/envs/dsde-cedt/lib/python3.11/site-packages/langchain/memory/__init__.py:33: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain.memory.buffer import (
/opt/homebrew/Caskroom/miniconda/base/envs/dsde-cedt/lib/python3.11/site-packages/pydantic/v1/validators.py:767: UserWarning: Mixing V1 and V2 models is not supported. `BaseLanguageModel` is a V2 model.
  warn(f'Mixing V1 and V2 models is not supported. `{type_.__name__}` is a V2 model.', UserWarnin

RuntimeError: no validator found for <class 'langchain_core.language_models.base.BaseLanguageModel'>, see `arbitrary_types_allowed` in Config

In [4]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [5]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)
print(ai_msg.content)

I translated "I love programming" into French.  The French translation is "J'adore la programmation."



In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)


# Define the funtion that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = model.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [7]:
config = {"configurable": {"thread_id": "abc123"}}

In [12]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob, it's nice to meet you! How can I help you today?


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name is Bob.


In [16]:
query = "What's my name?"
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob.


In [11]:
input_messages

[HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='8bc014e9-c114-4e0c-8308-67ba4767558c')]

In [17]:
state = app.get_state(config).values

for message in state["messages"]:
    print(message.content)

What's my name?
I do not know your name. As a large language model, I have no access to personal information about you unless you provide it to me within our current conversation.

Hi! I'm Bob.
Hi Bob, it's nice to meet you! How can I help you today?

What's my name?
You told me your name is Bob.

What's my name?
Your name is Bob.

What's my name?
Your name is Bob.



In [13]:
state = app.get_state(config).values

state

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='8bc014e9-c114-4e0c-8308-67ba4767558c'),
  AIMessage(content='I do not know your name. As a large language model, I have no access to personal information about you unless you provide it to me within our current conversation.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-014ed989-f785-4f95-a1dc-66c2b48c41ea-0', usage_metadata={'input_tokens': 23, 'output_tokens': 34, 'total_tokens': 57, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='2a888887-1f7a-4e52-91f8-2f510481ad80'),
  AIMessage(content="Hi Bob, it's nice to meet you! How can I help you today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_rat